In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model= "llama3.2:1b")

In [3]:
from langchain_core.prompts import PromptTemplate

prompt_template =PromptTemplate(
    template = "What is the capital of {country}?",
    input_variables = ["country"],
)

prompt = prompt_template.invoke({"country":"France"})

print(prompt)

llm.invoke(prompt_template.invoke({"country":"France"}))


text='What is the capital of France?'


AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-09-05T13:28:56.001019Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1247882000, 'load_duration': 1107708625, 'prompt_eval_count': 32, 'prompt_eval_duration': 94756917, 'eval_count': 8, 'eval_duration': 44937958, 'model_name': 'llama3.2:1b'}, id='run--9f450be5-587f-49df-9eca-170bdea78e2c-0', usage_metadata={'input_tokens': 32, 'output_tokens': 8, 'total_tokens': 40})

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

message_list=[
    SystemMessage(content="You are a helpful assistant that can answer questions about the capital of countries."),
    HumanMessage(content="What is the capital of South Korea?"),
    AIMessage(content="The capital of South Korea is Seoul."),
    HumanMessage(content="What is the capital of South Korea?")
]
llm.invoke(message_list)


AIMessage(content="The capital of South Korea is actually Pyongyang, which is also the capital of North Korea. However, I can tell you that Seoul is the de facto capital and the country's administrative center.", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-09-05T13:28:56.33396Z', 'done': True, 'done_reason': 'stop', 'total_duration': 322412292, 'load_duration': 51463125, 'prompt_eval_count': 74, 'prompt_eval_duration': 26448916, 'eval_count': 39, 'eval_duration': 243811375, 'model_name': 'llama3.2:1b'}, id='run--7f0ac23a-490a-4667-9b97-7e7e15ad85eb-0', usage_metadata={'input_tokens': 74, 'output_tokens': 39, 'total_tokens': 113})

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

chat_prompt_template =ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful assistant that can answer questions about the capital of countries."
    ),
    (
        "human",
        "What is the capital of {country}?"
    ),

])

chat_prompt = chat_prompt_template.invoke({"country":"France"})
print(chat_prompt)
print("==================")
ai_message = llm.invoke(chat_prompt_template.invoke({"country":"France"}))
print(ai_message)
print("==================")
output_parser = StrOutputParser()

answer = output_parser.invoke(llm.invoke(prompt_template.invoke({"country":"France"})))




messages=[SystemMessage(content='You are a helpful assistant that can answer questions about the capital of countries.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={})]
content='The capital of France is Paris.' additional_kwargs={} response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-09-05T13:32:20.829362Z', 'done': True, 'done_reason': 'stop', 'total_duration': 191419375, 'load_duration': 65361875, 'prompt_eval_count': 47, 'prompt_eval_duration': 80942958, 'eval_count': 8, 'eval_duration': 44596208, 'model_name': 'llama3.2:1b'} id='run--21aefe04-8ca8-4e72-9041-5dec88dacf66-0' usage_metadata={'input_tokens': 47, 'output_tokens': 8, 'total_tokens': 55}


In [14]:
ai_message

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-09-05T13:32:20.829362Z', 'done': True, 'done_reason': 'stop', 'total_duration': 191419375, 'load_duration': 65361875, 'prompt_eval_count': 47, 'prompt_eval_duration': 80942958, 'eval_count': 8, 'eval_duration': 44596208, 'model_name': 'llama3.2:1b'}, id='run--21aefe04-8ca8-4e72-9041-5dec88dacf66-0', usage_metadata={'input_tokens': 47, 'output_tokens': 8, 'total_tokens': 55})

In [15]:
answer

'The capital of France is Paris.'

In [28]:
#아래보다 다음과 같은방식이 더 정석이다 / JsonOutputParser는 사용하지않는다
from pydantic import BaseModel, Field

class CountryDetail(BaseModel):
    capital: str = Field(description="The capital of the country")
    population: int = Field(description="The population of the country")
    area: float = Field(description="The area of the country")
    currency: str = Field(description="The currency of the country")
    language: str = Field(description="The language of the country")

structured_llm = llm.with_structured_output(CountryDetail)

In [29]:
from langchain_core.output_parsers import JsonOutputParser

country_detail_prompt=PromptTemplate(
    template="""Given following information about {country}:
    What is the capital of {country}?
    What is the population of {country}?
    What is the area of {country}?
    What is the currency of {country}?
    What is the language of {country}?
    What is the capital of {country}?
    What is the population of {country}?

    return it in JSON format and return the JSON dictionary only
    """,
    input_variables=["country"]
)

# country_detail_prompt.invoke({"country":"France"})

# output_parser = JsonOutputParser()
json_ai_message = structured_llm.invoke(country_detail_prompt.invoke({"country":"France"}))
# output_parser.invoke(json_ai_message)

In [30]:
json_ai_message

CountryDetail(capital='Paris', population=67, area=643.0, currency='Euro', language='French')

In [31]:
json_ai_message.capital

'Paris'

In [32]:
json_ai_message.population

67

In [33]:
json_ai_message.model_dump()

{'capital': 'Paris',
 'population': 67,
 'area': 643.0,
 'currency': 'Euro',
 'language': 'French'}

In [34]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

chat_prompt_template =ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful assistant that can answer questions about the capital of countries."
    ),
    (
        "human",
        "What is the capital of {country}?"
    ),

])

chat_prompt = chat_prompt_template.invoke({"country":"France"})
print(chat_prompt)
print("==================")
ai_message = llm.invoke(chat_prompt_template.invoke({"country":"France"}))
print(ai_message)
print("==================")
output_parser = StrOutputParser()

answer = output_parser.invoke(llm.invoke(prompt_template.invoke({"country":"France"})))




messages=[SystemMessage(content='You are a helpful assistant that can answer questions about the capital of countries.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={})]
content='The capital of France is Paris.' additional_kwargs={} response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-09-05T13:53:10.94183Z', 'done': True, 'done_reason': 'stop', 'total_duration': 711019292, 'load_duration': 571628708, 'prompt_eval_count': 47, 'prompt_eval_duration': 94087583, 'eval_count': 8, 'eval_duration': 44529958, 'model_name': 'llama3.2:1b'} id='run--41937fda-f5e6-4b8a-a80b-870d9e561a75-0' usage_metadata={'input_tokens': 47, 'output_tokens': 8, 'total_tokens': 55}


In [ ]:
chain = 